In [ ]:
import pandas as pd

In [ ]:
def lemma_tokens(input_path):
  with open(input_path, 'r') as f:
    out_str = f.readlines()
    f.close()
  origin = []
  lemma = []
  pos = []
  idx = 0
  while idx < len(out_str):
    if out_str[idx].startswith("# text = "):
      idx += 1
      lemma_tmp = []
      pos_tmp = []
      orig_tmp = []
      while out_str[idx] != '\n':
        origi_i = out_str[idx].split('\t')[1].lower()
        lemma_i = out_str[idx].split('\t')[2]
        pos_tag = out_str[idx].split('\t')[3]
        #print(lemma_i)
        orig_tmp.append(origi_i.replace('#', ''))
        lemma_tmp.append(lemma_i.replace('#', ''))
        pos_tmp.append(pos_tag)
        idx += 1
      origin.append(orig_tmp)
      lemma.append(lemma_tmp)
      pos.append(pos_tmp)
    else:
      idx += 1
  return lemma

In [ ]:
sets = 'KIR'

label_path = '../data/processed_data/{}/label.txt'.format(sets)
txt_path = '../data/processed_data/{}/full_sample_anno.txt'.format(sets)
write_text_path = '../data/weak_data/{}/full_text.txt'.format(sets)
write_label_path = '../data/weak_data/{}/label.txt'.format(sets)
lemma_path = '../data/lemma_data/{}/full_sample.conllu'.format(sets)
label_list = []
with open(label_path, 'r') as f:
    for i in f.readlines():
        label_list.append(i.strip().split(';'))

token_list = []
with open(txt_path, 'r') as f:
    for i in f.readlines():
        token_list.append(i.strip().split(';')) 
        
lemma_list = []
with open(lemma_path, 'r') as f:
    lemma_list = lemma_tokens(lemma_path)

In [ ]:
len(lemma_list) == len(token_list)
for i in range(len(token_list)):
    assert len(lemma_list[i]) == len(token_list[i])
    assert len(lemma_list[i]) == len(label_list[i])

In [ ]:
def refine_label_text(sets):
    label_path = '../data/processed_data/{}/label.txt'.format(sets)
    txt_path = '../data/processed_data/{}/full_sample_anno.txt'.format(sets)
    write_text_path = '../data/weak_data/{}/full_text.txt'.format(sets)
    write_label_path = '../data/weak_data/{}/label.txt'.format(sets)
    lemma_path = '../data/lemma_data/{}/full_sample.conllu'.format(sets)
    stop_words_path = '../data/weak_data/stop_words.txt'
    label_list = []
    with open(label_path, 'r') as f:
        for i in f.readlines():
            label_list.append(i.strip().split(';'))

    token_list = []
    with open(txt_path, 'r') as f:
        for i in f.readlines():
            token_list.append(i.strip().split(';')) 
            
    stop_list = []
    with open(stop_words_path, 'r') as f:
        for i in f.readlines():
            stop_list.append(i.strip()) 
        f.close()

    new_label_list = []
    new_token_list = []
    count = 0
    for idx_1 in range(len(label_list)):
        label_row = label_list[idx_1]
        token_row = token_list[idx_1]
        new_label_row = []
        new_token_row = []
        for idx_2 in range(len(label_row)):
            label = label_row[idx_2]
            token = token_row[idx_2]
            if ' ' in token:
                token_ = token.split(' ')
                for j in range(len(token_)):
                    if token_[j] in stop_list:
                        new_label_row.append('O')
                        count +=1
                        #print('--')
                    else:
                        new_label_row.append(label)
                new_token_row += token_
            else:
                if token in stop_list:
                    new_label_row.append('O')
                    count +=1
                    #print('--')
                else:
                    new_label_row.append(label)
                new_token_row.append(token)
        assert len(new_label_row) == len(new_token_row)
        new_label_list.append(new_label_row)
        new_token_list.append(new_token_row)

    flat_new_label_list = [i for item in new_label_list for i in item]
    flat_new_token_list = [i for item in new_token_list for i in item]

    txt_string = ''
    label_string = ''
    for i in range(len(flat_new_token_list)):
        token = flat_new_token_list[i]
        label = flat_new_label_list[i]

        txt_string += token
        txt_string += ' '
        label_string += label
        label_string += ' '
        if token == '.':
            txt_string += '\n'
            label_string += '\n'

    with open(write_text_path, 'w+') as f:
        f.write(txt_string)
        f.close()

    with open(write_label_path, 'w+') as f:
        f.write(label_string)
        f.close()
        
    # check the length of labels and sequence
    lb = []
    tx = []
    with open(write_text_path, 'r') as f:
        for i in f.readlines():
            lb.append(i.strip().split(' '))

    with open(write_label_path, 'r') as f:
        for i in f.readlines():
            tx.append(i.strip().split(' ')) 
            
    assert len(lb) == len(tx)
    for i in range(len(lb)):
        assert len(lb[i]) == len(tx[i])
    print(count)

In [ ]:
sets = 'KIR'
# KIR SAD RTG OPER LAH 
refine_label_text(sets)

In [ ]:
sets = 'RTG'
# KIR SAD RTG OPER LAH 
refine_label_text(sets)

In [ ]:
sets = 'SAD'
# KIR SAD RTG OPER LAH 
refine_label_text(sets)

In [ ]:
sets = 'LAH'
# KIR SAD RTG OPER LAH 
refine_label_text(sets)

In [ ]:
sets = 'OPER'
# KIR SAD RTG OPER LAH 
refine_label_text(sets)